In [21]:
#Modelo 1 
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from scipy.optimize import curve_fit
import plotly.graph_objects as go

# 1. Descargar datos históricos del activo
activo = "AAPL"  # Ticker del activo
periodo = "1y"
datos = yf.Ticker(activo).history(period=periodo)

# 2. Calcular los retornos promedio (Average) y la desviación estándar (std)
datos['Average'] = datos[['High', 'Low']].mean(axis=1).pct_change()
datos['std'] = datos['Average'].rolling(window=5).std()

# Eliminar valores NaN para los ajustes
datos.dropna(inplace=True)

# --- MODELO POLINOMIAL ---
# Ajustar un modelo polinomial de grado 2
X_poly = np.column_stack((datos['Average'], datos['Average']**2))  # [x, x^2]
X_poly = sm.add_constant(X_poly)  # Añadir intercepto
modelo_poly = sm.OLS(datos['std'], X_poly).fit()

# Coeficientes del modelo polinomial
alpha_poly = modelo_poly.params[0]
beta_1_poly = modelo_poly.params[1]
beta_2_poly = modelo_poly.params[2]

# Predicciones del modelo polinomial
datos['std_pred_poly'] = alpha_poly + beta_1_poly * datos['Average'] + beta_2_poly * datos['Average']**2

# --- MODELO EXPONENCIAL ---
# Definir la función exponencial
def modelo_exponencial(x, alpha, beta):
    return alpha * np.exp(beta * x)

# Ajustar el modelo exponencial
popt, _ = curve_fit(modelo_exponencial, datos['Average'], datos['std'], maxfev=10000)
alpha_exp, beta_exp = popt

# Predicciones del modelo exponencial
datos['std_pred_exp'] = modelo_exponencial(datos['Average'], alpha_exp, beta_exp)

# --- GRAFICAR LOS RESULTADOS ---
fig = go.Figure()

# Datos observados
fig.add_trace(go.Scatter(x=datos['Average'], y=datos['std'],
                         mode='markers', name='Datos Observados',
                         marker=dict(size=8, color='blue', opacity=0.7)))

# Línea del modelo polinomial
x_vals = np.linspace(datos['Average'].min(), datos['Average'].max(), 100)
y_vals_poly = alpha_poly + beta_1_poly * x_vals + beta_2_poly * x_vals**2
fig.add_trace(go.Scatter(x=x_vals, y=y_vals_poly, mode='lines', name='Modelo Polinomial (Grado 2)',
                         line=dict(color='red')))

# Línea del modelo exponencial
y_vals_exp = modelo_exponencial(x_vals, alpha_exp, beta_exp)
fig.add_trace(go.Scatter(x=x_vals, y=y_vals_exp, mode='lines', name='Modelo Exponencial',
                         line=dict(color='green')))

# Configuración del gráfico
fig.update_layout(title="Modelos Polinomial y Exponencial | Desv. Estándar vs Retornos Promedio",
                  xaxis_title="Retornos Promedio (Average)",
                  yaxis_title="Desviación Estándar (std)",
                  template="plotly_white")

fig.show()

# --- RESULTADOS ---
print("Resultados del Modelo Polinomial:")
print(f"Intercepto (α): {alpha_poly:.4f}")
print(f"Coeficiente Lineal (β1): {beta_1_poly:.4f}")
print(f"Coeficiente Cuadrático (β2): {beta_2_poly:.4f}")
print(f"Resumen del Modelo Polinomial:\n{modelo_poly.summary()}")

print("\nResultados del Modelo Exponencial:")
print(f"Intercepto (α): {alpha_exp:.4f}")
print(f"Coeficiente Exponencial (β): {beta_exp:.4f}")


C:\Users\retir\AppData\Local\Temp\ipykernel_980\2765941249.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha_poly = modelo_poly.params[0]
C:\Users\retir\AppData\Local\Temp\ipykernel_980\2765941249.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta_1_poly = modelo_poly.params[1]
C:\Users\retir\AppData\Local\Temp\ipykernel_980\2765941249.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta_2_poly = modelo_poly.par

Resultados del Modelo Polinomial:
Intercepto (α): 0.0103
Coeficiente Lineal (β1): -0.0086
Coeficiente Cuadrático (β2): 5.0676
Resumen del Modelo Polinomial:
                            OLS Regression Results                            
Dep. Variable:                    std   R-squared:                       0.166
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     24.15
Date:                Mon, 27 Jan 2025   Prob (F-statistic):           2.74e-10
Time:                        20:23:46   Log-Likelihood:                 879.03
No. Observations:                 245   AIC:                            -1752.
Df Residuals:                     242   BIC:                            -1742.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t

In [38]:
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from scipy.optimize import curve_fit
import plotly.graph_objects as go    
from scipy.optimize import fsolve

stock = "AAPL"  # Ticker del activo
periodo = "1y"
datos = yf.Ticker(stock).history(period=periodo)

# 2. Calcular los retornos promedio (Average) y la desviación estándar (std)
datos['Average'] = datos[['High', 'Low']].mean(axis=1).pct_change()
datos['std'] = datos['Average'].rolling(window=5).std()
datos_filtrados = datos[datos['Average'] > 0]

# Eliminar valores NaN para los ajustes
datos.dropna(inplace=True)

# --- MODELO POLINOMIAL ---
# Ajustar un modelo polinomial de grado 2
X_poly = np.column_stack((datos['Average'], datos['Average']**2))  # [x, x^2]
X_poly = sm.add_constant(X_poly)  # Añadir intercepto
modelo_poly = sm.OLS(datos['std'], X_poly).fit()

# Coeficientes del modelo polinomial
alpha_poly = modelo_poly.params[0]
beta_1_poly = modelo_poly.params[1]
beta_2_poly = modelo_poly.params[2]

# Predicciones del modelo polinomial
datos['std_pred_poly'] = alpha_poly + beta_1_poly * datos['Average'] + beta_2_poly * datos['Average']**2

# --- MODELO EXPONENCIAL ---
# Definir la función exponencial
def modelo_exponencial(x, alpha, beta):
      return alpha * np.exp(beta * x)

# Ajustar el modelo exponencial
popt, _ = curve_fit(modelo_exponencial, datos['Average'], datos['std'], maxfev=10000)
alpha_exp, beta_exp = popt

# Predicciones del modelo exponencial
datos['std_pred_exp'] = modelo_exponencial(datos['Average'], alpha_exp, beta_exp)

# Crear un diccionario con los datos
data1 = { 'Average': datos['Average'].tolist(), 
        'std_pred_poly': datos['std_pred_poly'].tolist(), 
        'std_pred_exp': datos['std_pred_exp'].tolist() }

# Función para encontrar la intersección entre dos funciones
def interseccion(x):
    std_poly = alpha_poly + beta_1_poly * x + beta_2_poly * x**2
    std_exp = alpha_exp * np.exp(beta_exp * x)
    return std_poly - std_exp

# Valores iniciales para buscar las raíces
valores_iniciales = [-0.05, 0.05]  # Suponiendo dos intersecciones

# Resolver las raíces
intersecciones = [fsolve(interseccion, x0)[0] for x0 in valores_iniciales]

# Calcular las desviaciones estándar en las intersecciones
resultados = [(x, alpha_poly + beta_1_poly * x + beta_2_poly * x**2) for x in intersecciones]

# Mostrar los resultados
Intersección = {}
for i, (x, y) in enumerate(resultados):
    Intersección = {'Average': f"{x:.4f}", 'Risk': f"{y:.4f}"}
    print(f"Intersección {i + 1}: x = {x:.4f}, std = {y:.4f}")
    # Crear un diccionario con las intersecciones
    intersecciones_dict = {
        'Average': [x for x, y in resultados],
        'Risk': [y for x, y in resultados]
    }
if intersecciones_dict:
    print(f"""Las siguientes intersecciones son {intersecciones_dict}.
    El retorno mínimo es {float(intersecciones_dict['Average'][0] * 100).__round__(4)}% y el retorno máximo es {float(intersecciones_dict['Average'][1] * 100).__round__(4)}% 
    con un riesgo máximo de {float(intersecciones_dict['Risk'][1] * 100).__round__(4)}% y riesgo mínimo de {float(intersecciones_dict['Risk'][0] * 100).__round__(4)}%""")
else:
    print("No se encontraron intersecciones")








Intersección 1: x = -0.0077, std = 0.0107
Intersección 2: x = 0.0219, std = 0.0125
Las siguientes intersecciones son {'Average': [-0.007746381680267313, 0.021867886488545485], 'Risk': [0.010657894197817028, 0.012518963337939152]}.
    El retorno mínimo es -0.7746% y el retorno máximo es 2.1868% 
    con un riesgo máximo de 1.2519% y riesgo mínimo de 1.0658%


C:\Users\retir\AppData\Local\Temp\ipykernel_4040\1972207429.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha_poly = modelo_poly.params[0]
C:\Users\retir\AppData\Local\Temp\ipykernel_4040\1972207429.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta_1_poly = modelo_poly.params[1]
C:\Users\retir\AppData\Local\Temp\ipykernel_4040\1972207429.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta_2_poly = modelo_poly.